In [1]:
import pandas as pd
import sqlite3

## connection to the database

In [2]:
con = sqlite3.connect("../data/checking-logs.sqlite")

## create a new table datamart

 ## using Pandas methods, create two dataframes: test and control

## Close connection

In [3]:
con.close()